# Controlling accesss to attributes

* Following blocks are one possible implementation of vectors of `double`s.

* Here, member variable `new_name` is in `protected:` part.
* Member methods and subclass members can access this variable but from the outside of the class, we cannot access it.
* We call it **encapsulation**; instead of directly reading or writing to the variable, we would use mutator or reader **methods**.
* This is because to modularize software components to the level of integrated circuit chips.

``` C++
// Begin vector_double.h

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>


#ifndef LOG
#define LOG
#endif

#ifndef LOGBRACKET
// #define LOGBRACKET
#endif

#ifndef VECTOR_DOUBLE
#define VECTOR_DOUBLE

class RowVector 
{
    // automatic allocation
    // https://stackoverflow.com/questions/8553464/vector-as-a-class-member
    std::vector<double> columns;

    protected:
        std::string name;

    public:
        // Default constructor
		RowVector();

        ~ RowVector();

        // Default arguments
        // If the function could not find the argument in the call, it uses the default value.
        RowVector(const uint32_t n, const double *values=NULL, std::string new_name="None");

        RowVector(const uint32_t n, std::string new_name="None");

        RowVector(const RowVector & other);

        double & operator [] (const uint32_t i);

        double operator [] (const uint32_t i) const;

        const std::string get_name() const;

        RowVector operator + (const RowVector & other);

        RowVector operator * (const double a);

        const double operator * (const RowVector & other);

        void show();

        void resize(std::size_t new_size);

        std::size_t size() const noexcept;

        RowVector & operator += (const RowVector & other);

        RowVector & operator *= (const double a);
};

#endif
// End vector_double.h

```

``` C++
// Begin vector_double.cpp

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"

RowVector::RowVector(){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector()" << '\n';
#endif
    name = "None";
}


RowVector::~ RowVector(){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "~ RowVector()" << '\n';
#endif
}


RowVector::RowVector(const uint32_t n, const double *values, std::string new_name){
#ifdef LOG
    std::cout << '[' << &columns << ']' 
    << "RowVector(" << n << ", " << values << ", " << new_name << ")\n";
#endif
    columns.resize(n);

    // If initial values available, copy
    if (values){
        for (uint32_t i = 0; columns.size() > i; ++i){
            columns[i] = values[i];
        }
    }
    // If no initial values, set all values zero
    else{
        for (uint32_t i = 0; columns.size() > i; ++i){
            columns[i] = 0.0;
        }
    }

    name = new_name;
}


// Instead of implementing another constructor, calling an existing one
// c++ 11 or later
RowVector::RowVector(const uint32_t n, std::string new_name) : RowVector(n, NULL, new_name){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector(" << n << ", " << new_name << ")\n";
#endif
}


RowVector::RowVector(const RowVector & other){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector(" << & other << ")\n";
#endif
    // https://codereview.stackexchange.com/questions/149669/c-operator-overloading-for-matrix-operations-follow-up
    // http://www.cplusplus.com/reference/vector/vector/resize/
    columns.resize(other.columns.size());

    // element loop
    for(uint32_t i=0; columns.size() > i; ++i){
        columns[i] = other.columns[i];
    }

    // Copy name of the other one
    name = other.name;
    // Then append
    name.append("2");
}


double & RowVector::operator [] (const uint32_t i){
#ifdef LOGBRACKET
    std::cout << '[' << &columns << ']' << "double & RowVector::operator [] (" << i << ")\n";
#endif
    // Return reference; otherwise, unable to assign
    return columns[i];
}

double RowVector::operator [] (const uint32_t i) const {
#ifdef LOGBRACKET
    std::cout << '[' << &columns << ']' << "double RowVector::operator [] (" << i << ") const\n";
#endif
    // Return reference; otherwise, unable to assign
    return columns[i];
}


const std::string RowVector::get_name() const{
#ifdef LOG
    std::cout << '[' << &columns << ']' << "const std::string RowVector::get_name()\n";
#endif
    // Return constant; to prevent change
    return name;
}


RowVector RowVector::operator + (const RowVector & other){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector RowVector::operator + (" << & other << ")\n";
#endif
    // Check size
    assert(columns.size() == other.columns.size());

    // Make a new vector to return
    RowVector temp(other);

    // Element loop
    for (uint32_t i=0; columns.size() > i; ++i){
        temp[i] += columns[i];
    }

    // Returning a temporary image
    return temp;
}


RowVector RowVector::operator * (const double a){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector RowVector::operator * (" << a << ")\n";
#endif

    // Make a new vector to return
    RowVector temp(*this);

    // Element loop in `for each` style
    // c++ 11 or later
    for (auto & element : temp.columns){
        element *= a;
    }

    // Returning a temporary image
    return temp;
}


const double RowVector::operator * (const RowVector & other){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "const double RowVector::operator * (" << & other << ")\n";
#endif

    // Check size
    assert(columns.size() == other.columns.size());

    double dot_product = 0.0;

    // Element loop
    for (uint32_t i = 0; columns.size() > i; ++i){
        dot_product += columns[i] * other.columns[i];
    }

    // Returning a temporary image
    return dot_product;
}


void RowVector::show(){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "void RowVector::show()\n";
#endif
    for (uint32_t i=0; columns.size()> i; ++i){
        std::cout << name << '[' << i << "] = " << columns[i] << '\n';
    }
}


void RowVector::resize(std::size_t new_size){
#ifdef LOG
    std::cout << '[' << &columns << ']' << "void RowVector::resize(" << new_size << ")\n";
#endif
    columns.resize(new_size);
}


std::size_t RowVector::size() const noexcept{
#ifdef LOG
    std::cout << '[' << &columns << ']' << "std::size_t RowVector::size() const noexcept\n";
#endif
    return columns.size();
}


RowVector & RowVector::operator += (const RowVector & other) {
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector & RowVector::operator += (" << & other << ")\n";
#endif
    // https://stackoverflow.com/questions/4581961/c-how-to-overload-operator
    for (uint32_t i=0; size()>i; ++i){
        columns[i] += other[i];
    }
    return *this;
}


RowVector & RowVector::operator *= (const double a) {
#ifdef LOG
    std::cout << '[' << &columns << ']' << "RowVector & RowVector::operator *= (" << a << ")\n";
#endif
    // https://stackoverflow.com/questions/4581961/c-how-to-overload-operator
    for (uint32_t i=0; size()>i; ++i){
        columns[i] *= a;
    }
    return *this;
}

// End vector_double.cpp
// Build command : g++ -Wall -g -std=c++14 vector_double.cpp -fsyntax-only

```

``` C++
// Begin cpp_vector_double_practice.cpp

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"

int32_t main(int32_t argn, char *argv[]){
	double s[] = {1.0, 2.0};

    std::cout << "RowVector row (2u, s, \"row\");\n";
	RowVector row (2u, s, "row");

    row.show();

    std::cout << "RowVector another_row (row);\n";
	RowVector another_row (row);
    row.show();
    another_row.show();

    std::cout << "another_row[1] += 0.5;\n";
    another_row[1] += 0.5;
    row.show();
    another_row.show();

    std::cout << "RowVector row_plus_another(row + another_row);\n";
    RowVector row_plus_another(row + another_row);
    row.show();
    another_row.show();
    row_plus_another.show();

    std::cout << "RowVector zeros(3);\n";
	RowVector zeros(3u, "zeros");
    row.show();
    another_row.show();
    row_plus_another.show();
    zeros.show();

    double t[] = {2.0, -1.0};
	RowVector ortho (2u, t, "ortho");
    double dot = row * ortho;
    std::cout << "double dot = row * ortho;\n";
    std::cout << "dot  = " << dot << '\n';

    std::cout << "dot = row * row;\n";
    dot = row * row;
    std::cout << "dot  = " << dot << '\n';

}

// End cpp_vector_double_practice.cpp
// Build command : g++ -Wall -g -std=c++14 cpp_vector_double_practice.cpp vector_double.cpp -o cpp_vector_double_practice

``` 

* In the mean while, following code blocks depict a possible implementation in python.

In [ ]:
import collections


class Vector(collections.UserList):

    def __add__(self, other):

        # check size
        assert len(self) == len(other), f"Lengths are different ({len(self)} == {len(other)})"

        # trying list comprehension
        return Vector([a + b for a, b in zip(self, other)])

    def __radd__(self, other):
        # What is this?

        return self.__add__(other)

    def __mul__(self, other):

        if isinstance(other, (int, float, complex)):
            result = Vector([a * other for a in self])
        elif isinstance(other, Vector):
            assert len(self) == len(other),  f"Lengths are different ({len(self)} == {len(other)})"
            result = sum(a * b for a, b in zip(self, other))
        
        return result

    def __rmul__(self, other):
        return __mul__(self, other)
    
    def __str__(self):
        return '\n'.join(f"{hex(id(self))}[{i}] = {self[i]}" for i in range(len(self)))



In [ ]:
print("a = Vector([1, 2])")
a = Vector([1, 2])
print(a)

print("b = Vector(a)")
b = Vector(a)
print(a)
print(b)

print("b[1] += (-0.5)")
b[1] += (-0.5)
print(a)
print(b)

print("c = a + b")
c = a + b
print(a)
print(b)
print(c)

print("ortho = Vector([2, -1])")
ortho = Vector([2, -1])
print(a)
print(b)
print(c)
print(ortho)

print("dot = a * ortho")
dot = a * ortho
print(f"a * ortho = {dot}")

print("dot = a * a")
dot = a * a
print(f"a * a = {dot}")



# Matrix class example

## In Python

* Following code blocks are a possible implementation of matrix in python.
* As in C++ example, it will build on the prior `Vector` class.

In [ ]:
import collections
import copy


class Matrix(collections.UserList):
    def __init__(self, m, n=None, values=None):
        if values is not None:
            self.m = int(m) # number of rows
            self.n = int(n) # number of columns
            # Again utilizing Vector class and list comprehension
            self.data = [Vector(values[(i * n):((i+1) * n)]) for i in range(m)]

        elif n is None:
            if isinstance(m, Matrix):
                # copy constructor
                self.m = m.m
                self.n = m.n
                # To avoid referencing rows of m matrix
                self.data = copy.deepcopy(m.data)
            elif isinstance(m, Vector):
                # Vector to n x 1 Matrix
                self.data = [Vector([value]) for value in m]
                self.m = len(self.data)
                self.n = 1
                


In [ ]:
matA = Matrix(2, 2, list(range(4)))
print(matA)

matB = Matrix(matA)
matB[0][0] = matA[0][0] + 7
print(matA)
print(matB)
assert matA[0][0] != matB[0][0], "Please use deep copy"

vecC = Vector([1, 0])
matC = Matrix(vecC)
print(matA)
print(matB)
print(matC)



## In C++

* Following code blocks present a possible implementation of matrix class in C++.
* Please note that to build these files, `vector_double.h` and `vector_double.cpp` files are necessary.



```C++
// Begin matrix_double.h

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"

#ifndef MATRIX_DOUBLE
#define MATRIX_DOUBLE

class Matrix
{
    std::vector<RowVector> rows;

    protected:
        std::string name;

    public:
		Matrix();

        ~ Matrix();

        Matrix(const uint32_t m, const uint32_t n, const double *values, std::string new_name="None");

        Matrix(const uint32_t m, const uint32_t n, std::string new_name="None");

        Matrix(const Matrix & other, std::string new_name="");

        Matrix(const RowVector & other, std::string new_name="");

        RowVector & operator [] (const uint32_t i);

        const RowVector operator [] (const uint32_t i) const;

        const std::string get_name() const;

        Matrix operator + (const Matrix & other);

        Matrix operator * (const double a);

        RowVector operator * (const RowVector &v);

        Matrix operator * (const Matrix & other);

        void show();

        Matrix transpose();

        const size_t get_height() const;

        const size_t get_width() const;
};

#endif
// End matrix_double.h
```

``` C++
// Begin matrix_double.cpp

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"
#include    "matrix_double.h"


Matrix::Matrix(){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix()" << '\n';
#endif
    name = "None";
}


Matrix::~ Matrix(){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "~ Matrix()" << '\n';
#endif
}


Matrix::Matrix(const uint32_t m, const uint32_t n, const double *values, std::string new_name){
#ifdef LOG
    std::cout << '[' << &rows << ']' 
    << "Matrix(" << m << ", "<< n << ", " << values << ", " << new_name << ")\n";
#endif
    name = new_name;

    rows.resize(m);

    // If initial values available, copy
    if (values){
        // row loop
        for (uint32_t i = 0; m > i; ++i){
            rows[i].resize(n);
            // column loop
            for (uint32_t j = 0; n > j; ++j){
                rows[i][j] = *(values + i * n + j) ;
            }
        }
    }
    // If no initial values, set all values zero
    else{
        // row loop
        for (uint32_t i = 0; m > i; ++i){
            rows[i].resize(n);
            // column loop
            for (uint32_t j = 0; n > j; ++j){
                rows[i][j] = 0.0;
            }
        }
    }

}

// Instead of implementing another constructor, calling an existing one
// c++ 11 or later
Matrix::Matrix(const uint32_t m, const uint32_t n, std::string new_name) : Matrix(m, n, NULL, new_name){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix(" << m << ", " << n << ", " << new_name << ")\n";
#endif
}


Matrix::Matrix(const Matrix & other, std::string new_name){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix(" << & other << ")\n";
#endif
    // https://codereview.stackexchange.com/questions/149669/c-operator-overloading-for-matrix-operations-follow-up
    // http://www.cplusplus.com/reference/vector/vector/resize/
    rows.resize(other.rows.size());
    // row loop
    for(uint32_t i=0; rows.size() > i; ++i){
        rows[i].resize(other.rows[i].size());

        // column loop
        for(uint32_t j=0; other.rows[i].size() > j; ++j){
            // Another possibility is as follows
            // rows[i][j] = other.rows[i][j];
            // However for now the line above would create a temporary row vector
            // To avoid seemingly unnecessary such temporary object, 
            // for now would use the following line
            rows[i][j] = other.rows[i][j];
        }

    }

    if ("" != new_name){
        name = new_name;
    }
    else{
        // Copy name of the other one
        name = other.name;
        // Then append
        name.append("2");
    }
}


Matrix::Matrix(const RowVector & other, std::string new_name){
    // RowVector -> n x 1 matrix    
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix(const RowVector &" << & other << ")\n";
#endif
    rows.resize(other.size());

    // row loop
    for(uint32_t i=0; rows.size() > i; ++i){
        rows[i].resize(1);
        rows[i][0] = other[0];
    }

    if ("" != new_name){
        name = new_name;
    }
    else{
        // Copy name of the other one
        name = other.get_name();
        // Then append
        name.append("2");
    }
}


RowVector & Matrix::operator [] (const uint32_t i){
#ifdef LOGBRACKET
    std::cout << '[' << &rows << ']' << "RowVector & Matrix::operator [] (" << i << ")\n";
#endif
    // Return reference; otherwise, unable to assign
    return rows[i];
}

const RowVector Matrix::operator [] (const uint32_t i) const {
#ifdef LOGBRACKET
    std::cout << '[' << &rows << ']' << "const RowVector Matrix::operator [] (" << i << ")\n";
#endif
    // Return reference; otherwise, unable to assign
    return rows[i];
}


const std::string Matrix::get_name() const{
#ifdef LOG
    std::cout << '[' << &rows << ']' << "const std::string Matrix::get_name()\n";
#endif
    // Return constant; to prevent change
    return name;
}


Matrix Matrix::operator + (const Matrix & other){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix Matrix::operator + ("<< & other <<")\n";
#endif
    // Check size
    assert(this->get_height() == other.get_height());
    assert(this->get_width() == other.get_width());

#ifdef LOG
    std::cout << "Matrix temp(other);\n";
#endif
    // Make a new vector to return
    Matrix temp(other, get_name() + '+' + other.get_name());

#ifdef LOG
    std::cout << "Begin row loop\n";
#endif
    // Row loop
    for (uint32_t i=0; rows.size() > i; ++i){
        temp[i] += rows[i];
    }
#ifdef LOG
    std::cout << "End row loop\n";
#endif

    // Returning a temporary image
    return temp;
}


Matrix Matrix::operator * (const double a){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix Matrix::operator * (" << a << ")\n";
#endif

    // Make a new vector to return
    // https://stackoverflow.com/questions/332111/how-do-i-convert-a-double-into-a-string-in-c
    Matrix temp(*this, std::to_string(a) + '*' + get_name());

    // Element loop in `for each` style
    // c++ 11 or later
    for (auto & element : temp.rows){
        element *= a;
    }

    // Returning a temporary image
    return temp;
}


RowVector Matrix::operator * (const RowVector &v){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix Matrix::operator * (" << &v << ")\n";
#endif

    // Make a new vector to return
    RowVector temp(rows.size(), NULL, name + '*' + v.get_name());

    // Element loop in `for each` style
    // c++ 11 or later
    for (uint32_t i=0; rows.size()>i; ++i){
        temp[i] = rows[i] * v;
    }

    // Returning a temporary image
    return temp;
}


Matrix Matrix::operator * (const Matrix & other){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix Matrix::operator * (" << &other << ")\n";
#endif

    // Check size
    assert(rows[0].size() == other.rows.size());

    Matrix temp(rows.size(), other[0].size(), name + '*' + other.name);

    // row loop
    for (uint32_t i = 0; rows.size() > i; ++i){
        // column loop
        for(uint32_t j = 0; other[0].size() > j; ++j){
            // dummy index loop
            for(uint32_t k = 0; rows[0].size() > k; ++k){
                temp[i][j] += rows[i][k] * other[k][j];
            }
        }
    }

    // Returning a temporary image
    return temp;
}


void Matrix::show(){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "void Matrix::show()\n";
#endif
    // row loop
    for (uint32_t i=0; rows.size()> i; ++i){
        // column loop
        for (uint32_t j=0; rows[i].size()> j; ++j){
            std::cout << get_name() << '['<< i << "][" << j << "]= " << rows[i][j] << '\n';
        }
    }
}


Matrix Matrix::transpose(){
#ifdef LOG
    std::cout << '[' << &rows << ']' << "Matrix Matrix::transpose()\n";
#endif
    Matrix temp(rows[0].size(), rows.size(), name+"T");

    // row loop
    for(uint32_t i=0; temp.rows.size()> i; ++i){
        // column loop
        for(uint32_t j=0; temp.rows.size()> j; ++j){
            temp[i][j] = rows[i][j];
        }        
    }

    return temp;
}


const size_t Matrix::get_height() const{
    return rows.size();
}


const size_t Matrix::get_width() const{
    return rows[0].size();
}


// End matrix_double.cpp
// Build command : g++ -Wall -g -std=c++14 matrix_double.cpp -fsyntax-only

```

``` C++
// Begin cpp_matrix_double_practice.cpp

#include <cassert>
#include <cmath>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "matrix_double.h"

int32_t main(int32_t argn, char *argv[]){
	double s[] = {1.0, 0.0,
                  0.0, 1.0};

    std::cout << "Matrix id (2u, 2u, s, \"identity\");\n";
	Matrix identity (2u, 2u, s, "id");

    identity.show();

    double r[] = {+cos(M_PI/6.0), sin(M_PI/6.0),
                  -sin(M_PI/6.0), cos(M_PI/6.0)};

    std::cout << "Matrix rotation (2u, 2u, r, \"rot\");\n";
    Matrix rotation (2u, 2u, r, "rot");
    identity.show();
    rotation.show();

    std::cout << "Matrix sum(identity + rotation);\n";
    Matrix sum(identity + rotation);
    identity.show();
    rotation.show();
    sum.show();

    // Check sum operation result
    for (uint32_t i=0; 2u > i; ++i){
        for (uint32_t j=0; 2u > j; ++j){
            assert(sum[i][j] == (identity[i][j] + rotation[i][j]));
        }
    }

    std::cout << "Matrix twice(identity * 2.0);\n";
    Matrix twice(identity * 2.0);

    // Check scala multiplication result
    assert(twice[0][0] == 2.0);
    assert(twice[0][1] == 0.0);
    assert(twice[1][0] == 0.0);
    assert(twice[1][1] == 2.0);

    std::cout << "Matrix new_axis(twice * rotation);\n";
    Matrix new_axis(twice * rotation);

    // Check matrix multiplication result
    for (uint32_t i=0; 2u > i; ++i){
        for (uint32_t j=0; 2u > j; ++j){
            assert(new_axis[i][j] == (2.0 * rotation[i][j]));
        }
    }

    Matrix ninety_degrees(rotation * rotation * rotation);

    // Check matrix multiplication result
    assert(abs(ninety_degrees[0][0] - ( 0.0)) < 1e-12);
    assert(abs(ninety_degrees[0][1] - ( 1.0)) < 1e-12);
    assert(abs(ninety_degrees[1][0] - (-1.0)) < 1e-12);
    assert(abs(ninety_degrees[1][1] - ( 0.0)) < 1e-12);

    // State Space Representation Ax + B u
    double xi_d[] = {1.0, 0.0};
    double ones_d[] = {1.0, 1.0};

    Matrix xi(2, 1, xi_d, "xi");
    Matrix B(2, 1, ones_d, "B");

    double u = 0.75;

    Matrix xj;
    // xj = A xi  + B u
    xj = rotation * xi + B * u;

    xj.show();

    assert(abs(xj[0][0] - ( 0.75 + cos(M_PI/6.0))) < 1e-12);
    assert(abs(xj[1][0] - ( 0.75 - sin(M_PI/6.0))) < 1e-12);

}

// End cpp_matrix_double_practice.cpp
// Build command : g++ -Wall -g -std=c++14 cpp_matrix_double_practice.cpp vector_double.cpp matrix_double.cpp -o cpp_matrix_double_practice
```

* The build command above lists necessary files.



# State Space Representation Example

## C++

* Again, this example builds on top of the `Matrix` and `RowVector` examples.

``` C++
// Begin lti_dt.h

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"
#include    "matrix_double.h"


#ifndef LTI_DT

// Discrete Time State Space model
class LTI_DT{
    protected:
        Matrix A;
        Matrix B;
        Matrix C;
        Matrix D;
        Matrix X;

        size_t m, n;

    public:
        LTI_DT(Matrix &new_A, Matrix &new_B, Matrix &new_C, Matrix &new_D, Matrix &new_X);
        ~LTI_DT();
        const Matrix get_y(const double u);
        void get_next_x(const double u);
};
#endif

// End lti_dt.h

```

``` C++
// Begin lti_dt.cpp

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"
#include    "matrix_double.h"
#include    "lti_dt.h"

// Discrete Time State Space model
LTI_DT::LTI_DT(Matrix &new_A, Matrix &new_B, Matrix &new_C, Matrix &new_D, Matrix &new_X){
#ifdef LOG
    std::cout << '[' << &A << ']' << "LTI_DT::LTI_DT(" << &new_A << ", " << &new_B << ", " << &new_C << ", " << &new_D << ")\n";
#endif
    
#ifdef LOG
    std::cout << "LTI_DT::LTI_DT(): A = new_A;\n";
#endif
    A = new_A;    
#ifdef LOG
    std::cout << "LTI_DT::LTI_DT(): B = new_B;\n";
#endif
    B = new_B;    
#ifdef LOG
    std::cout << "LTI_DT::LTI_DT(): C = new_C;\n";
#endif
    C = new_C;    
#ifdef LOG
    std::cout << "LTI_DT::LTI_DT(): D = new_D;\n";
#endif
    D = new_D;    
#ifdef LOG
    std::cout << "LTI_DT::LTI_DT(): X = new_X;\n";
#endif
    X = new_X;

    // is A matrix square?
    assert(A.get_height() == A.get_width());

    // number of state variables
    n = A.get_height();

    // check number of rows of B matrix
    assert(B.get_height() == n);
    
    // expected size of input
    m = B.get_width();

}

LTI_DT::~LTI_DT(){
#ifdef LOG
    std::cout << '[' << &A << ']' << "LTI_DT::!LTI_DT()\n";
#endif
#ifdef LOG
    std::cout << "delete &A;\n";
#endif
    // delete &A;
#ifdef LOG
    std::cout << "delete &B;\n";
#endif
    // delete &B;
#ifdef LOG
    std::cout << "delete &C;\n";
#endif
    // delete &C;
#ifdef LOG
    std::cout << "delete &D;\n";
#endif
    // delete &D;
#ifdef LOG
    std::cout << "delete &X;\n";
#endif
    // delete &X;
}

const Matrix LTI_DT::get_y(const double u){
    return Matrix (C * X + D * u);
}

void LTI_DT::get_next_x(const double u){
    Matrix next_X (A * X + B * u);

    // delete &X;

    X = next_X;
}

// End lti_dt.cpp

```

``` C++
// Begin lti_dt_example.cpp

#include <cassert>
#include <cstdint>
#include <exception>
#include <iostream>
#include <string>
#include <vector>

#include    "vector_double.h"
#include    "matrix_double.h"
#include    "lti_dt.h"

int32_t main(int32_t argn, char *argv[]){
    // https://ccrma.stanford.edu/~jos/fp/State_Space_Simulation_Matlab.html

    const double A_d[] = {0, 1, -1, 0};
    std::cout << "Matrix A(2u, 2u, A_d, \"A\");\n";
    Matrix A(2u, 2u, A_d, "A");

    const double B_d[] = {0, 1};
    std::cout << "Matrix B(2u, 1u, B_d, \"B\");\n";
    Matrix B(2u, 1u, B_d, "B");

    const double C_d[] = {1, 0, 0, 1, 0, 1};
    std::cout << "Matrix C(3u, 2u, C_d, \"C\");\n";
    Matrix C(3u, 2u, C_d, "C");

    const double D_d[] = {0, 0, 0};
    std::cout << "Matrix D(3u, 1u, D_d, \"D\");\n";
    Matrix D(3u, 1u, D_d, "D");

    const uint32_t n = 10;
    const double u[n] = {1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0};

    std::vector<Matrix> y_list;

    Matrix X(2u, 1u, "x");

    LTI_DT ss_dt(A, B, C, D, X);

    for(uint32_t k=0; n > k; ++k){
        Matrix y_now(ss_dt.get_y(u[k]));
        y_list.push_back(y_now);
        ss_dt.get_next_x(u[k]);
    }

    for(uint32_t i=0; n>i; ++i){
        std::cout << "y[" << i << "] = " << y_list[i][2][0] << '\n';
    }

    return 0;
}
// End lti_dt_example.cpp
// Build command : g++ -Wall -g -std=c++14 lti_dt_example.cpp vector_double.cpp matrix_double.cpp lti_dt.cpp -o lti_dt_example


```

* However, this example may have some obvious problem.  What do you think?

* For python implementation, please refer to another file.